In [3]:
import pandas as pd

df = pd.read_parquet('../allocine_spider.parquet')
df.head()

,actors,critics_score,date,director,editor,french_boxoffice,french_first_week_boxoffice,french_visa,genre,langage,length,nationality,title,url,viewers_score,vo_title
0,Amruta Subhash|Archit Deodhar|Parth Bhalerao,3.5,2015-10-07,Avinash Arun,Les Films du Préau,1958,735,142936,Aventure|Drame,Tamoul,107,Inde,La Forteresse,/article/fichearticle_gen_carticle=18646392.html,3.3,Killa
1,Yvan Attal|Sophie Quinton|Nicole Garcia,3.5,2012-03-14,Lucas Belvaux,Diaphana Distribution,270549,104634,127961,Drame,Français,104,France,38 témoins,/diaporamas/cinema/diaporama-18712695/,2.6,None
2,Thure Lindhardt|Zachary Booth|Julianne Nicholson,3.5,2012-08-22,Ira Sachs,KMBO,13267,5908,133557,Drame,Danois|Anglais,101,U.S.A.,Keep the Lights On,/article/fichearticle_gen_carticle=18616043.html,3.4,None
3,Rita Blanco|Joaquim de Almeida|Roland Giraud,3.5,2013-04-24,Ruben Alves,Pathé Films,1197974,288750,124489,Comédie,Anglais|Français,91,Portugal|France,La Cage Dorée,/video/player_gen_cmedia=19498177&cfilm=109860...,3.9,None
4,Jessica Chastain|Jason Clarke|Joel Edgerton,4.1,2013-01-23,Kathryn Bigelow,Universal Pictures International France,524533,237214,135377,Action|Thriller,Anglais,149,U.S.A.,Zero Dark Thirty,/video/player_gen_cmedia=19433296&cfilm=193444...,3.9,None


In [4]:
df.dtypes

actors                          object
critics_score                  float64
date                            object
director                        object
editor                          object
french_boxoffice                 int64
french_first_week_boxoffice      int64
french_visa                     object
genre                           object
langage                         object
length                           int64
nationality                     object
title                           object
url                             object
viewers_score                  float64
vo_title                        object
dtype: object

In [9]:
df['date']= pd.to_datetime(df['date'])

In [10]:
df['genre'] = df['genre'].str.split('|')
df['actors'] = df['actors'].str.split('|')
df['nationality'] = df['nationality'].str.split('|')
df['langage'] = df['langage'].str.split('|') 	

In [11]:
feature_of_interest = [
    'actors',
    'critics_score',
    'date',
    'director',
    'editor',
    'genre',
    'langage',
    'length',
    'nationality',
    'viewers_score'
]

info_film = ['french_visa', 'title', 'vo_title', 'url']


target = 'french_first_week_boxoffice'
data = feature_of_interest
data.append(target)

data.extend(info_film)
df_to_train = df[data]

In [12]:
df_to_train

,actors,critics_score,date,director,editor,genre,langage,length,nationality,viewers_score,french_first_week_boxoffice,french_visa,title,vo_title,url
0,"[Ami Tôma, Mana Ashida, Yûki Kaji]",3.5,2023-09-06,Keiichi Hara,Eurozoom,"[Animation, Drame, Fantastique]",[Japonais],116,[Japon],3.9,20761,160405,Le Château Solitaire dans le Miroir,Kagami no Kojou,/video/player_gen_cmedia=19602173&cfilm=311559...
1,"[Bárbara Lennie, Irene Escolar, Itziar Manero]",3.5,2023-11-29,Itsaso Arana,Arizona Distribution,[Comédie dramatique],[Espagnol],85,[Espagne],3.0,2991,160976,Les Filles vont bien,Las chicas están bien,/article/fichearticle_gen_carticle=1000099469....
2,None,3.5,2024-07-10,Rashin Kheyrieh,Les Films du Whippet,"[Animation, Famille]",[Silencieux],38,"[Iran, Russie]",2.8,106,2024002054,L'Arbre à Contes,None,/article/fichearticle_gen_carticle=1000089882....
3,"[Colman Domingo, Clarence Maclin, Sean San Jose]",3.5,2025-01-29,Greg Kwedar,Metropolitan FilmExport,[Drame],[Anglais],107,[U.S.A.],3.9,24945,163554,Sing Sing,None,/article/fichearticle_gen_carticle=1000131867....
4,"[Fanny Ardant, Mathieu Kassovitz, Laetitia Dosch]",3.5,2024-03-13,Thierry Klifa,Apollo Films,"[Comédie dramatique, Policier]",[Français],116,[France],2.9,38177,152501,Les Rois de la Piste,None,/article/fichearticle_gen_carticle=1000070726....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,"[Emilie Lan Dürr, Florence Miailhe, Maxime Gémin]",4.2,2021-09-29,Florence Miailhe,Gebeka Films,"[Animation, Drame]",[Français],84,[France],4.0,5764,121662,La Traversée,None,/article/fichearticle_gen_carticle=18705601.html
498,"[Kotone Furukawa, Ayumu Nakajima, Hyunri]",4.4,2022-04-06,Ryūsuke Hamaguchi,Diaphana Distribution,"[Drame, Romance]",[Japonais],121,[Japon],3.8,55324,155941,Contes du hasard et autres fantaisies,Guzen to sozo,/diaporamas/cinema/diaporama-18712206/
499,"[Yûya Endô, Kanji Tsuda, Yuya Matsuura]",4.4,2021-07-21,Arthur Harari,Le Pacte,"[Drame, Historique, Guerre]",[Japonais],167,"[France, Japon, Allemagne, Belgique, Italie, C...",4.0,14592,144549,Onoda - 10 000 nuits dans la jungle,Onoda,/article/fichearticle_gen_carticle=1000035796....
500,"[Alana Haim, Cooper Hoffman, Bradley Cooper]",4.5,2022-01-05,Paul Thomas Anderson,Universal Pictures International France,"[Comédie, Drame, Romance]",[Anglais],134,[U.S.A.],3.7,116871,156124,Licorice Pizza,None,/video/player_gen_cmedia=19594999&cfilm=257754...


In [14]:
# from azure.ai.ml.entities import Data
# from azure.ai.ml.constants import AssetTypes
import pyarrow as pa
import pyarrow.parquet as pq

table = pa.Table.from_pandas(df_to_train)
pq.write_table(table, 'allocine_spider_clean.parquet')
# new_data_asset = Data(
#     path='allocine_spider.parquet',
#     type=AssetTypes.URI_FILE,
#     name="allo_cine", 
#     version="1.1",              # nouvelle version
#     description="Premier jet pour entrainer les modèles"
# )

# ml_client.data.create_or_update(new_data_asset)